<a href="https://colab.research.google.com/github/KhondamirRustamov/AF2seq/blob/main/ProteinMPNN_fixbb1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title setup
%%time
import os
if not os.path.isdir("params"):
  # get code
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.1")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar")
  os.system("tar -xf alphafold_params_2022-12-06.tar -C params")

os.system('''pip install "colabfold[alphafold] @ git+https://github.com/sokrypton/ColabFold"''')
#os.system('''pip install -q "jax[cuda]==0.3.25" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html''')
#pip install -q "jax[cuda]==0.3.25" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from IPython.display import HTML
from google.colab import files
import numpy as np


CPU times: user 1.63 s, sys: 395 ms, total: 2.02 s
Wall time: 2min 36s


In [2]:
#@title setup1
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:08
🔁 Restarting kernel...


In [1]:
#@title setup1.1
!mamba install openmm==7.7.0 pdbfixer -c conda-forge


Looking for: ['openmm==7.7.0', 'pdbfixer']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64   1%
conda-forge/noarch    ⣾  [+] 0.2s
conda-forge/linux-64  16%
conda-forge/noarch    14%[+] 0.3s
conda-forge/linux-64  25%
conda-forge/noarch    36%[+] 0.4s
conda-forge/linux-64  34%
conda-forge/noarch    57%[+] 0.5s
conda-forge/linux-64  44%
conda-forge/noarch    67%[+] 0.6s
conda-forge/linux-64  48%
conda-forge/noarch    77%[+] 0.7s
conda-forge/linux-64  53%
conda-forge/noarch    99%conda-forge/noarch                                
[+] 0.8s
conda-forge/linux-64  67%[+] 0.9s
conda-forge/linux-64  76%[+] 1.0s
conda-forge/linux-64  95%conda-forge/linux-64                              

Pinned packages:
  - python 3.10.*
  - python 3.10.*
  - python_abi 3.10.* *cp310*
  - cuda-version 12.*


Transaction

  Prefix: /usr/local

  Updating specs:

   - openmm==7.7.0
   - pdbfixer
   - ca-certificates
   - certifi
   - openssl


  Package               Version  Build                Channel           Size
─

In [2]:
#@title Functions
from alphafold.relax import relax
from alphafold.relax import utils
from alphafold.common import protein as p_cf

from colabdesign.af.alphafold.common import protein
from colabdesign.shared.protein import renum_pdb_str
from colabdesign.af.alphafold.common import residue_constants

import pdbfixer
import numpy as np
import jax

def rank_array(input_array):
    # numpy.argsort returns the indices that would sort an array.
    # We convert it to a python list before returning
    return list(np.argsort(input_array))

def rank_and_write_pdb(af_model, name, write_all=False, renum_pdb = True):

    ranking = rank_array(af_model.aux['all']['loss'])
    if write_all != True:
        ranking = [ranking[0]]

    aux = af_model._tmp["best"]["aux"]
    aux = aux["all"]

    p = {k:aux[k] for k in ["aatype","residue_index","atom_positions","atom_mask"]}
    p["b_factors"] = 100 * p["atom_mask"] * aux["plddt"][...,None]

    def to_pdb_str(x, n=None):
        p_str = protein.to_pdb(protein.Protein(**x))
        p_str = "\n".join(p_str.splitlines()[1:-2])
        if renum_pdb: p_str = renum_pdb_str(p_str, af_model._lengths)
        if n is not None:
            p_str = f"MODEL{n:8}\n{p_str}\nENDMDL\n"
        return p_str

    m=1
    for n in ranking:
        p_str = ""
        p_str += to_pdb_str(jax.tree_map(lambda x:x[n],p), n+1)
        p_str += "END\n"

        with open(name + '_model_{n}_rank_{m}.pdb'.format(n=n, m=m), 'w') as f:
            f.write(p_str)
        m+=1

MODRES = {'MSE':'MET','MLY':'LYS','FME':'MET','HYP':'PRO',
          'TPO':'THR','CSO':'CYS','SEP':'SER','M3L':'LYS',
          'HSK':'HIS','SAC':'SER','PCA':'GLU','DAL':'ALA',
          'CME':'CYS','CSD':'CYS','OCS':'CYS','DPR':'PRO',
          'B3K':'LYS','ALY':'LYS','YCM':'CYS','MLZ':'LYS',
          '4BF':'TYR','KCX':'LYS','B3E':'GLU','B3D':'ASP',
          'HZP':'PRO','CSX':'CYS','BAL':'ALA','HIC':'HIS',
          'DBZ':'ALA','DCY':'CYS','DVA':'VAL','NLE':'LEU',
          'SMC':'CYS','AGM':'ARG','B3A':'ALA','DAS':'ASP',
          'DLY':'LYS','DSN':'SER','DTH':'THR','GL3':'GLY',
          'HY3':'PRO','LLP':'LYS','MGN':'GLN','MHS':'HIS',
          'TRQ':'TRP','B3Y':'TYR','PHI':'PHE','PTR':'TYR',
          'TYS':'TYR','IAS':'ASP','GPL':'LYS','KYN':'TRP',
          'CSD':'CYS','SEC':'CYS'}

def pdb_to_string(pdb_file, chains=None, models=[1]):
  '''read pdb file and return as string'''

  if chains is not None:
    if "," in chains: chains = chains.split(",")
    if not isinstance(chains,list): chains = [chains]
  if models is not None:
    if not isinstance(models,list): models = [models]

  modres = {**MODRES}
  lines = []
  seen = []
  model = 1
  for line in open(pdb_file,"rb"):
    line = line.decode("utf-8","ignore").rstrip()
    if line[:5] == "MODEL":
      model = int(line[5:])
    if models is None or model in models:
      if line[:6] == "MODRES":
        k = line[12:15]
        v = line[24:27]
        if k not in modres and v in residue_constants.restype_3to1:
          modres[k] = v
      if line[:6] == "HETATM":
        k = line[17:20]
        if k in modres:
          line = "ATOM  "+line[6:17]+modres[k]+line[20:]
      if line[:4] == "ATOM":
        chain = line[21:22]
        if chains is None or chain in chains:
          atom = line[12:12+4].strip()
          resi = line[17:17+3]
          resn = line[22:22+5].strip()
          if resn[-1].isalpha(): # alternative atom
            resn = resn[:-1]
            line = line[:26]+" "+line[27:]
          key = f"{model}_{chain}_{resn}_{resi}_{atom}"
          if key not in seen: # skip alternative placements
            lines.append(line)
            seen.append(key)
      if line[:5] == "MODEL" or line[:3] == "TER" or line[:6] == "ENDMDL":
        lines.append(line)
  return "\n".join(lines)

def relax_me(pdb_in, pdb_out):
  pdb_str = pdb_to_string(pdb_in)
  protein_obj = p_cf.from_pdb_string(pdb_str)
  amber_relaxer = relax.AmberRelaxation(
    max_iterations=0,
    tolerance=2.39,
    stiffness=10.0,
    exclude_residues=[],
    max_outer_iterations=3,
    use_gpu=False)
  relaxed_pdb_lines, _, _ = amber_relaxer.process(prot=protein_obj)
  with open(pdb_out, 'w') as f:
      f.write(relaxed_pdb_lines)


def rank_array_predict(input_array):
    # numpy.argsort returns the indices that would sort an array.
    # We convert it to a python list before returning
    return list(np.argsort(input_array))[::-1]

def rank_and_write_pdb_predict(af_model, name, write_all=False, renum_pdb = True):
    ranking = rank_array_predict(np.mean(af_model.aux['all']['plddt'],-1))
    if write_all != True:
        ranking = [ranking[0]]

    aux = af_model.aux
    aux = aux["all"]

    p = {k:aux[k] for k in ["aatype","residue_index","atom_positions","atom_mask"]}
    p["b_factors"] = 100 * p["atom_mask"] * aux["plddt"][...,None]

    def to_pdb_str(x, n=None):
        p_str = protein.to_pdb(protein.Protein(**x))
        p_str = "\n".join(p_str.splitlines()[1:-2])
        if renum_pdb: p_str = renum_pdb_str(p_str, af_model._lengths)
        #if n is not None:
        #    p_str = f"MODEL{n:8}\n{p_str}\nENDMDL\n"
        return p_str

    m=1

    pdbs_out = []

    for n in ranking:
        p_str = ""
        p_str += to_pdb_str(jax.tree_map(lambda x:x[n],p), n+1)
        p_str += "END\n"

        with open(name + '_model_{n}_rank_{m}.pdb'.format(n=n, m=m), 'w') as f:
            f.write(p_str)
        pdbs_out.append(name + '_model_{n}_rank_{m}.pdb'.format(n=n, m=m))
        m+=1
    return pdbs_out


aa_order = residue_constants.restype_order
order_aa = {b:a for a,b in aa_order.items()}

def get_best_seq(aux):
    x = aux["seq"]["hard"].argmax(-1)
    return ["".join([order_aa[a] for a in s]) for s in x]

In [27]:
#@title Run ProteinMPNN
from colabdesign.mpnn import mk_mpnn_model
import glob
import jax
import jax.numpy as jnp
import os

from colabdesign import mk_afdesign_model

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from tqdm.notebook import tqdm
import glob

from colabdesign.af.alphafold.common import protein
from colabdesign.shared.protein import renum_pdb_str
from colabdesign.af.alphafold.common import residue_constants

chain_to_design = 'A'  #@param {type:"string"}
design_poses = '51,53,60,66,69,72'  #@param {type:"string"}
design_poses = design_poses.split(',')
design_pos = ','.join([f'{chain_to_design}{i}' for i in design_poses])
number_of_seqs = 100 #@param {type:"integer"}
pdb_filename = '9lyz_lysocyme.pdb' #@param {type:"string"}
model_name = "v_48_030"
results_directory = 'results_full' #@param {type:"string"}

mpnn_model = mk_mpnn_model(model_name, backbone_noise=0.0)
mpnn_model.prep_inputs(pdb_filename=pdb_filename,
                       chain=chain_to_design,
                       fix_pos=design_pos,
                       inverse=True)
os.system(f'mkdir {results_directory}/mpnn/')
samples = mpnn_model.sample_parallel(temperature=0.2, batch=number_of_seqs)
name = 'Lysozyme' #@param {type:"string"}
with open(f'{results_directory}/mpnn/{name}.pickle', 'wb') as handle:
    pickle.dump(samples, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
#@title Read sequences from ProteinMPNN
sequences = []
names = []

for file in glob.glob(f'{results_directory}/mpnn/*.pickle'):
    name = file.split('.pickle')[0].split('/')[-1]
    with open(file, 'rb') as f:
        x = pickle.load(f)
    n = 0
    for seq in x['seq']:
        if seq not in sequences:
            sequences.append(seq)
            names.append('{name}_{n}'.format(name=name, n=n))
            n += 1
        else:
            print('duplicate found!')

In [29]:
print(len(sequences))

100


In [30]:
#@title Predict sequences from ProteinMPNN

os.system(f'mkdir {results_directory}/mpnn/pdb')
os.system(f'mkdir {results_directory}/mpnn/pickle')
af_model = mk_afdesign_model(protocol="fixbb",
                             use_templates=True)
af_model.prep_inputs(pdb_filename=pdb_filename,
                     chain=chain_to_design,
                     rm_aa='C')

for i,x in enumerate(sequences):
  seq = sequences[i]
  name = names[i]
  #print('Predicting:', name, seq)
  af_model.set_seq(seq)
  af_model.predict(num_recycles=3)

  pdbs = rank_and_write_pdb_predict(af_model, name=f'{results_directory}/mpnn/pdb/' + name)
  '''
  print('Relaxing...')
  for pdb in pdbs:
    pdb_out = pdb.split('.pdb')[0] + '_relaxed.pdb'
    relax_me(pdb, pdb_out)
    os.remove(pdb) # delete unrelaxed pdb file
  '''
  with open(f'{results_directory}/mpnn/pickle/{name}.pickle'.format(name=name), 'wb') as handle:
    pickle.dump(af_model.aux['all'], handle, protocol=pickle.HIGHEST_PROTOCOL)

predict models [0] recycles 3 hard 1 soft 0 temp 1 seqid 0.64 loss 1.22 dgram_cce 1.22 plddt 0.94 ptm 0.87 rmsd 0.60


<ipython-input-2-d9d3135b2631>:153: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  p_str += to_pdb_str(jax.tree_map(lambda x:x[n],p), n+1)


predict models [0] recycles 3 hard 1 soft 0 temp 1 seqid 0.64 loss 1.27 dgram_cce 1.27 plddt 0.93 ptm 0.86 rmsd 0.63
predict models [0] recycles 3 hard 1 soft 0 temp 1 seqid 0.67 loss 1.15 dgram_cce 1.15 plddt 0.94 ptm 0.87 rmsd 0.53
predict models [0] recycles 3 hard 1 soft 0 temp 1 seqid 0.66 loss 1.23 dgram_cce 1.23 plddt 0.93 ptm 0.85 rmsd 0.57
predict models [0] recycles 3 hard 1 soft 0 temp 1 seqid 0.65 loss 1.21 dgram_cce 1.21 plddt 0.92 ptm 0.85 rmsd 0.58
predict models [0] recycles 3 hard 1 soft 0 temp 1 seqid 0.68 loss 1.23 dgram_cce 1.23 plddt 0.93 ptm 0.86 rmsd 0.65
predict models [0] recycles 3 hard 1 soft 0 temp 1 seqid 0.64 loss 1.19 dgram_cce 1.19 plddt 0.94 ptm 0.86 rmsd 0.54
predict models [0] recycles 3 hard 1 soft 0 temp 1 seqid 0.65 loss 1.24 dgram_cce 1.24 plddt 0.93 ptm 0.86 rmsd 0.59
predict models [0] recycles 3 hard 1 soft 0 temp 1 seqid 0.65 loss 1.71 dgram_cce 1.71 plddt 0.93 ptm 0.86 rmsd 1.58
predict models [0] recycles 3 hard 1 soft 0 temp 1 seqid 0.64 lo

In [18]:
#@title Predict results for WT (wild type)

wt_sequence = 'KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNRCKGTDVQAWIRGCRL' #@param {type:"string"}
seq = wt_sequence
name = 'wt'
print('Predicting:', name, seq)
af_model.set_seq(seq)
af_model.predict(num_recycles=3)

pdbs = rank_and_write_pdb_predict(af_model, name=f'{results_directory}/mpnn/pdb/' + name)

with open(f'{results_directory}/mpnn/pickle/{name}.pickle'.format(name=name), 'wb') as handle:
  pickle.dump(af_model.aux['all'], handle, protocol=pickle.HIGHEST_PROTOCOL)

Predicting: wt KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNRCKGTDVQAWIRGCRL
predict models [0] recycles 3 hard 1 soft 0 temp 1 seqid 1 loss 1.27 dgram_cce 1.27 plddt 0.95 ptm 0.86 rmsd 0.73


<ipython-input-2-d9d3135b2631>:153: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  p_str += to_pdb_str(jax.tree_map(lambda x:x[n],p), n+1)


In [24]:
#@title Read scores
names = []
plddts = []
ptms = []

for file in glob.glob(f'results/mpnn/pickle/*.pickle'):
    name = file.split('.pickle')[0].split('/')[-1]
    with open(file, 'rb') as f:
        x = pickle.load(f)
    n = 0
    names.append(name)
    plddts.append(np.mean(x['plddt'][0]))
    ptms.append(float(x['ptm']))

<ipython-input-24-89c63ca66c3d>:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ptms.append(float(x['ptm']))


In [25]:
#@title Save as table .csv
import pandas as pd
name_of_csv_file = 'my_results2' #@param {type:"string"}
df = {'name': names,
      'plddts':plddts,
      'ptm': ptms}
df = pd.DataFrame(df)
df.to_csv(f'{name_of_csv_file}.csv')


In [26]:
#@title Package and download results.
from google.colab import files
import os

results_zip = f"{results_directory}_pdb.zip"
os.system(f"zip -r {results_zip} results/mpnn/pdb/")
files.download(f"{results_zip}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>